## AUD vs PAG propotion


In [ ]:

omc_prop
omc_prop_select = omc_prop[(omc_prop["species"]=="MMus") | (omc_prop["species"]=="STeg")]
omc_prop_aud = omc_prop_select[(omc_prop_select['area']=="AUD")].reset_index(drop=True)
omc_prop_pag = omc_prop_select[(omc_prop_select['area']=="PAG")].reset_index(drop=True)
omc_prop_aud = omc_prop_aud.rename(columns=({'proportion':'aud_prop'}))
omc_prop_aud['pag_prop'] = omc_prop_pag['proportion']
plot_df = omc_prop_aud.copy().drop(["area", "type"], axis=1)
plot_df

sns.scatterplot(plot_df, x="aud_prop", y="pag_prop",hue="species",
                s=100)
plt.xlabel("AUD Proportion")
plt.ylabel("PAG Proportion")

plt.xlim(0,0.7)

plt.savefig(out_path+"omc_proportion_AUDvPAG2.svg", dpi=300, bbox_inches="tight")



# scatter and volcano plot for comparing resampled data


In [ ]:

# combine IT and PT
sp_stats_it = stvmm_calc_stats(plot_it, species=["STeg_resampled", "MMus_resampled"])
sp_stats_pt = stvmm_calc_stats(plot_pt, species=["STeg_resampled", "MMus_resampled"])

sp_stats = pd.concat([sp_stats_it,sp_stats_pt])
sp_stats


stvmm_area_scatter_type(sp_stats, title="OMC - Resampled", x="STeg_resampled", y="MMus_resampled")
# plt.savefig(out_path+"OMC_scatter_mapseq_resampled.jpg", dpi=300, bbox_inches="tight")
plt.show()


In [ ]:

## volcano plot
omc_prop = pd.concat([plot_it, plot_pt])
plot = stvmm_calc_ttest(omc_prop, sp1="MMus_resampled", sp2="STeg_resampled")

plot_volcano(plot, title="OMC MAPseq Proportion - Resampled", shape="type")
plt.xlim(-3,3)


# plt.savefig(out_path+"OMC_volcano_mapseq_resampled.jpg", dpi=300, bbox_inches="tight")


In [ ]:

# compare number of cells
omc_preprocess[0]

# make dataframe w/ # unique barcodes per animal

nBC_df = pd.DataFrame(columns=["Total Neurons", "IT Neurons", "CT Neurons", "PT Neurons",
                               "mice", "species", "dataset"])

for i in range(metadata.shape[0]):
    df = omc_preprocess[i]
    tot = df.shape[0]
    it = df[df['type']=="IT"].shape[0]
    ct = df[df['type']=="CT"].shape[0]
    pt = df[df['type']=="PT"].shape[0]

    row = [tot, it, ct, pt, metadata.loc[i,"mice"], metadata.loc[i,"species"], metadata.loc[i,"dataset"]]

    nBC_df.loc[i] = row

nBC_df
area_proportion_dot_plot(nBC_df, title="All Unique Barcodes - OMC",
                         to_plot="Total Neurons", add_legend=False)
plt.savefig(out_path+"uBC_OMC_all.jpg", dpi=300, bbox_inches="tight")
plt.show()

area_proportion_dot_plot(nBC_df, title="IT Unique Barcodes - OMC",
                         to_plot="IT Neurons", add_legend=False)
plt.show()

area_proportion_dot_plot(nBC_df, title="CT Unique Barcodes - OMC",
                         to_plot="CT Neurons", add_legend=False)
plt.show()

area_proportion_dot_plot(nBC_df, title="PT Unique Barcodes - OMC",
                         to_plot="PT Neurons", add_legend=False)
plt.show()




# PCA  to cluster animals


In [ ]:

it_df = omc_it_prop.pivot(index="mice", columns="area", values="proportion")
pt_df = omc_pt_prop.pivot(index="mice", columns="area", values="proportion")

all_df = pd.concat([it_df, pt_df], axis=1)
from sklearn.decomposition import PCA

pca = PCA()
principalComponents = pca.fit_transform(all_df)
explain_var = pca.explained_variance_ratio_

# plot pcs
plt.scatter(principalComponents[0:5,0], principalComponents[0:5,1], label="lab mouse")
plt.scatter(principalComponents[5:12,0], principalComponents[5:12,1], label="singing mouse")
plt.xlabel("PC1 "+ str(round(explain_var[0], 4)*100)+ "%")
plt.ylabel("PC2 "+ str(round(explain_var[1], 4)*100)+ "%")
plt.legend()
plt.show()
plt.savefig(out_path+"omc_prop_pca12.svg", dpi=300, bbox_inches="tight")
plt.scatter(principalComponents[0:5,1], principalComponents[0:5,2], label="lab mouse")
plt.scatter(principalComponents[5:12,1], principalComponents[5:12,2], label="singing mouse")
plt.xlabel("PC2 "+ str(round(explain_var[1], 4)*100)+ "%")
plt.ylabel("PC3 "+ str(round(explain_var[2], 4)*100)+ "%")
plt.legend()
plt.show()
plt.savefig(out_path+"omc_prop_pca23.svg", dpi=300, bbox_inches="tight")


In [ ]:

from mpl_toolkits import mplot3d

# plt.scatter(principalComponents[0:5,1], principalComponents[0:5,2], label="lab mouse")


# Creating figure
fig = plt.figure(figsize = (10, 7))
ax = plt.axes(projection ="3d")
 
# Creating plot
ax.scatter3D(principalComponents[0:5,0], principalComponents[0:5,1], principalComponents[0:5,2], label="lab mouse")
ax.scatter3D(principalComponents[5:12,0], principalComponents[5:12,1], principalComponents[5:12,2], label="singing mouse")
ax.set_xlabel("PC1 "+ str(round(explain_var[0], 4)*100)+ "%")
ax.set_ylabel("PC2 "+ str(round(explain_var[1], 4)*100)+ "%")
ax.set_zlabel("PC3 "+ str(round(explain_var[2], 4)*100)+ "%")
plt.legend()
plt.show()
components = pca.components_
plot_pca = pd.DataFrame(components, columns=all_df.columns).T
cols = ["PC"+str(x) for x in range(1,12)]
plot_pca.columns = cols
plot_pca.reset_index(inplace=True)

plt.plot(plot_pca["area"], plot_pca["PC1"], '-o', label="PC1 "+ str(round(explain_var[0], 4)*100)+ "%")
plt.plot(plot_pca["area"], plot_pca["PC2"], '-o', label="PC2 "+ str(round(explain_var[1], 4)*100)+ "%")
plt.plot(plot_pca["area"], plot_pca["PC3"], '-o', label="PC3 "+ str(round(explain_var[2], 4)*100)+ "%")
plt.legend()
plt.axhline(color="gray", linestyle="--")
plt.savefig(out_path+"pca_loadings.png", dpi=300, bbox_inches="tight")
plt.show()



In [ ]:

import plotly.express as px

plot_df = pd.DataFrame(principalComponents)
plot_df.index = all_df.index
cols = ["PC"+str(x) for x in range(1,12)]
plot_df.columns = cols
plot_df["species"] = "MMus"
plot_df.iloc[5:12,11] = "STeg"

fig = px.scatter_3d(plot_df, x="PC1", y="PC2", z="PC3",
                    color="species")
fig.show()
